# * Reconcile : Vinsight Data

## Parameter

In [1]:
import pandas as pd
import numpy as np
import configparser
import datetime as dt
import re
from functools import reduce
# import matplotlib as mpl
# from matplotlib import pyplot as plt
# from matplotlib.gridspec import GridSpec
# mpl.rcParams['axes.spines.top'] = False
# mpl.rcParams['axes.spines.right'] = False


''' Config '''

config = configparser.ConfigParser()
config.read('../my_config.ini')
config.sections()

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
op_file_name = f'Agg_Perf_Nationwide-{str_curr_dt}'
op_file_name

## Rawdata

In [3]:
''' Rawdata '''

# data_src = f"../data/raw/Agg_Perf_Nationwide-20240606.csv"
data_src = f"../data/raw/{op_file_name}.csv"
df = pd.read_csv(data_src, low_memory=False)

cols = ['TM_KEY_DAY', 'TM_KEY_WK', 'TM_KEY_MTH', 'TM_KEY_QTR', 'TM_KEY_YR', 'CENTER', 'METRIC_GRP', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME', 
        'ACTUAL_AS_OF', 'AGG_TYPE', 'RR_IND', 'GRY_IND', 'UOM', 'PERIOD', 'ACTUAL_SNAP', 'ACTUAL_AGG', 'TARGET_SNAP', 'TARGET_AGG', 'BASELINE_SNAP', 'BASELINE_AGG', 'PPN_TM']

df = df[cols]

''' Filters '''
df = df[df['TM_KEY_YR'] == 2024] # YEAR filter
# df = df[df['TM_KEY_MTH'] >= 202401] # MONTH filter
print(f'\ndf : {df.shape[0]} rows, {df.shape[1]} columns')


df : 226178 rows, 24 columns


In [4]:
df.tail(3)

,TM_KEY_DAY,TM_KEY_WK,TM_KEY_MTH,TM_KEY_QTR,TM_KEY_YR,CENTER,METRIC_GRP,PRODUCT_GRP,COMP_CD,METRIC_CD,...,GRY_IND,UOM,PERIOD,ACTUAL_SNAP,ACTUAL_AGG,TARGET_SNAP,TARGET_AGG,BASELINE_SNAP,BASELINE_AGG,PPN_TM
564428,20240609,2024023,202406,20242,2024,Revenue,Sales,Postpaid,DTAC,DB2S000100AE,...,GYR,subs,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-10 11:22:44
564429,20240609,2024023,202406,20242,2024,Revenue,Market Share,NaN,DTAC,VIN00062,...,GYR,subs,M,NaN,9.190509e+06,NaN,NaN,NaN,NaN,2024-06-10 11:22:44
564430,20240609,2024023,202406,20242,2024,Revenue,Revenue,All Services,ALL,B0R000100GEO,...,GYR,baht,M,NaN,2.822781e+09,NaN,2.819904e+09,NaN,0.0,2024-06-10 11:22:44


In [5]:
# df[['METRIC_GRP']].drop_duplicates().sort_values(['METRIC_GRP']).reset_index(drop=True)
# df[['METRIC_GRP', 'METRIC_CD', 'METRIC_NAME']].drop_duplicates().sort_values(['METRIC_GRP', 'METRIC_CD'])

# df[df['METRIC_NAME'].str.contains('Inflow M1')]
# df[(df['METRIC_NAME'].str.contains('Inflow M1')) & (df['COMP_CD'].isin(['TRUE', 'DTAC'])) & (~df['METRIC_CD'].str.contains('C$|H$|MCOM$|CORP$|GEO$|A[A-K]$'))]\
#     [['METRIC_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']].drop_duplicates().sort_values(['METRIC_GRP', 'COMP_CD', 'METRIC_CD']).reset_index(drop=True)

tmp_df = df
tmp_df = tmp_df[(tmp_df['PRODUCT_GRP']=='TOL') & (~tmp_df['METRIC_CD'].str.contains('[0-9]C$|[0-9]H$|[0-9]MCOM$|[0-9]CORP$|[0-9]GEO$|[0-9]A[A-K]$'))]
tmp_df = tmp_df[(tmp_df['METRIC_NAME'].str.contains('Gross Add'))] 
tmp_df = tmp_df[['METRIC_GRP', 'METRIC_CD', 'METRIC_NAME']].drop_duplicates().sort_values(['METRIC_GRP', 'METRIC_CD']).reset_index(drop=True)
tmp_df

,METRIC_GRP,METRIC_CD,METRIC_NAME
0,Sales,TB3S000100,TOL Gross Adds - Connected
1,Sales,TB3S000101,TOL Gross Adds - Registration
2,Sales,TB3S000102,TOL Gross Adds - Connected (Consumer)
3,Sales,TB3S000103,TOL Gross Adds - Connected (DataService)
4,Sales,TB3S000900,TOL Gross Adds - Connected (Installation)
5,Sales,TB3S000901,TOL Gross Adds - Connected (Installation - Con...
6,Sales,TB3S000902,TOL Gross Adds - Connected (Installation - Dat...


In [6]:
# df.to_excel(f'{op_dir}/{op_file}', sheet_name='Data', index=False)
tmp_df.to_excel(f'temp/tmp_df.xlsx', index=False)

In [24]:
# df[(df['METRIC_CD'].str.contains('DB1R000900|DB2R000500|TB1R000900|TB2R000500')) & (~df['METRIC_CD'].str.contains('C$|H$|MCOM$|CORP$|GEO$|A[A-K]$'))]\
#     [['METRIC_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']].drop_duplicates().sort_values(['METRIC_GRP', 'COMP_CD', 'METRIC_CD']).reset_index(drop=True)

In [49]:
''' Function '''

def view_group(grp, cd, name):
    grp = grp
    cd = cd
    name = name
    flag = ''

    # CORP & MCOM
    if re.search('C$|H$|MCOM$', cd) and (not re.search('A[A-K]$', cd)): flag = 'CORP & MCOM'
    elif re.search('CUS$', cd): flag = 'Cust Location'
    # Revenue
    elif grp == 'Revenue' and any(x in name for x in ['New Revenue', 'Existing Revenue']): flag = 'New/Existing'
    elif grp == 'Revenue' and any(x in name for x in ['Paid Amount', 'On Due', 'Overdue']): flag = 'Paid Amount'
    elif grp == 'Revenue' and any(x in name for x in ['Revenue']): flag = 'Revenue'
    # Sales
    elif grp == 'Sales' and any(x in name for x in ['Inflow M1']): flag = 'Inflow M1'
    elif grp == 'Sales' and any(x in name for x in ['Inflow M2']): flag = 'Inflow M2'
    elif grp == 'Sales' and any(x in name for x in ['Gross Add']): flag = 'Gross Adds'
    elif grp == 'Sales' and any(x in name for x in ['%AP']): flag = '%AP'
    elif grp == 'Sales' and any(x in name for x in ['AP 1D']): flag = 'AP 1D'
    elif grp == 'Sales' and any(x in name for x in ['AP In Month']): flag = 'AP MTH'
    elif grp == 'Sales' and any(x in name for x in ['Activation Subs']): flag = 'Activation'
    elif grp == 'Sales' and any(x in name for x in ['Conversion']): flag = '%Conversion'
    elif grp == 'Sales' and any(x in name for x in ['GA ARPU', 'GA RC']): flag = 'GA ARPU/RC'
    # Subs
    elif grp == 'Subs' and any(x in name for x in ['Net Add']): flag = 'Net Adds'
    elif grp == 'Subs' and any(x in name for x in ['%NAD']): flag = '%NAD'
    elif grp == 'Subs' and any(x in name for x in ['%M4']): flag = '%M4'
    elif grp == 'Subs' and any(x in name for x in ['Reported Sub']): flag = 'Reported Subs'
    elif grp == 'Subs' and any(x in name for x in ['Usage Subs', 'Active Caller', 'Active Subs']): flag = 'Active Subs'
    elif grp == 'Subs' and any(x in name for x in ['NAD']): flag = 'NAD'
    elif grp == 'Subs' and any(x in name for x in ['Revenue Subs']): flag = 'Rev Subs'
    # MKS
    elif grp == 'Market Share' and any(x in name for x in ['Broadband']): flag = '%BB MKS'
    elif grp == 'Market Share' and (not any(x in name for x in ['Broadband'])) & any(x in name for x in ['(Subs)']): flag = 'MB MKS(Subs)'
    elif grp == 'Market Share' and (not any(x in name for x in ['Broadband', '(Subs)'])): flag = '%MB MKS'
    # Churn
    elif grp == 'Retention & Churn' and any(x in name for x in ['Churn Subs']): flag = 'Churn Subs'
    elif grp == 'Retention & Churn' and any(x in name for x in ['Churn Rate']): flag = '%Churn Rate'
    # Others
    elif any(x in name for x in ['ARPU']): flag = 'ARPU'
    elif any(x in name for x in ['SubBase']): flag = 'SubBase'
    elif any(x in name for x in ['New Subs']): flag = 'New Subs'
    elif any(x in name for x in ['Silent']): flag = 'Silent'
    elif any(x in name for x in ['60DPD']): flag = '60DPD'
    elif any(x in name for x in ['Quality']): flag = 'Quality'
    else: flag = 'Unknown'
 
    return flag

In [50]:
# df_tmp['FLAG'] = df_tmp['METRIC_GRP'].apply(lambda x: group_flag(str(x)))
df['VIEW_GRP'] = df.apply(lambda x: view_group(str(x['METRIC_GRP']), str(x['METRIC_CD']), str(x['METRIC_NAME'])), axis=1)

# df.sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD'])

# df[df['METRIC_GRP']=='Revenue'].sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD'])
# df[df['METRIC_GRP']=='Market Share'].sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD'])
# df[df['METRIC_GRP']=='Retention & Churn'].sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD'])

# df[df['VIEW_GRP']=='Revenue'].sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD'])
# df[df['VIEW_GRP']=='Unknown'].sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD'])
# df[df['VIEW_GRP']!='CORP & MCOM'].sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD'])

df[~df['VIEW_GRP'].isin(['CORP & MCOM', 'Cust Location', 'Unknown'])].sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD']).reset_index(drop=True)

# df[['VIEW_GRP']].drop_duplicates().sort_values(['VIEW_GRP'])
# df[['METRIC_GRP', 'VIEW_GRP']].drop_duplicates().sort_values(['METRIC_GRP', 'VIEW_GRP'])

,TM_KEY_DAY,TM_KEY_WK,TM_KEY_MTH,TM_KEY_QTR,TM_KEY_YR,CENTER,METRIC_GRP,PRODUCT_GRP,COMP_CD,METRIC_CD,...,UOM,PERIOD,ACTUAL_SNAP,ACTUAL_AGG,TARGET_SNAP,TARGET_AGG,BASELINE_SNAP,BASELINE_AGG,PPN_TM,VIEW_GRP
0,20240101,2024001,202401,20241,2024,Revenue,Sales,Prepaid,ALL,B1S001000,...,%,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-07 11:55:57,%AP
1,20240102,2024001,202401,20241,2024,Revenue,Sales,Prepaid,ALL,B1S001000,...,%,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-07 11:55:57,%AP
2,20240103,2024001,202401,20241,2024,Revenue,Sales,Prepaid,ALL,B1S001000,...,%,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-07 11:55:57,%AP
3,20240104,2024001,202401,20241,2024,Revenue,Sales,Prepaid,ALL,B1S001000,...,%,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-07 11:55:57,%AP
4,20240105,2024001,202401,20241,2024,Revenue,Sales,Prepaid,ALL,B1S001000,...,%,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-07 11:55:57,%AP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177845,20240602,2024022,202406,20242,2024,Revenue,Subs,TOL,TRUE,TB3S020604,...,subs,M,3211196.0,3211196.0,NaN,NaN,NaN,NaN,2024-06-07 11:55:57,SubBase
177846,20240603,2024023,202406,20242,2024,Revenue,Subs,TOL,TRUE,TB3S020604,...,subs,M,3212049.0,3212049.0,NaN,NaN,NaN,NaN,2024-06-07 11:55:57,SubBase
177847,20240604,2024023,202406,20242,2024,Revenue,Subs,TOL,TRUE,TB3S020604,...,subs,M,3213114.0,3213114.0,NaN,NaN,NaN,NaN,2024-06-07 11:55:57,SubBase
177848,20240605,2024023,202406,20242,2024,Revenue,Subs,TOL,TRUE,TB3S020604,...,subs,M,NaN,3213114.0,NaN,NaN,NaN,NaN,2024-06-07 11:55:57,SubBase
